# Sistemas de Recomendación de filtrado colaborativo

En esta notebook, seguiremos con ***filtrado colaborativo***, una estrategia de recomendación basada en encontrar "entidades" similares. Existen dos alternativas:

* ***User-User collaborative filtering.***. Solo considera el comportamiento previo de los usuarios (por ejemplo, sus ratings). La idea es sencilla. Si a dos usuarios $U_{1}$ y $U_{2}$ les interesaron los elementos $I_{a}$ y $I_{b}$, pero al usuario $U_{2}$ también le interesó el elemento $I_{c}$ que todavía no fue visto por $U_{1}$, se podría asumir que $U_{1}$ podría estar interesado en $I_{c}$.

* ***Item-Item collaborative filtering.*** No considera las semejanzas entre los usuarios, sino que entre los elementos. En este contexto, las predicciones para un usuario $U$ nuevo que solo rateo pocos elementos, pueden ser fácilmente calculadas considerando los ratings que otros usuarios dieron a elementos similares.


<p align="center">
<img src="https://miro.medium.com/max/1920/1*QvhetbRjCr1vryTch_2HZQ.jpeg" width="600">
</p>


## User-user collaborative filtering

Nuevamente vamos a usar un [dataset](https://raw.githubusercontent.com/tommantonela/sistemasRecomendacion2019/master/User-User%20Collaborative%20Filtering%20-%20movie-row.csv) de películas. 
Este dataset contiene los ratings que $25$ usuarios $u$ le asignaron a $100$ películas $m$. Si el usuario $u$ no le asignó rating a la película $m$, la celda correspondiente estará vacía. 

In [0]:
import pandas as pd 
import numpy as np

url = 'https://raw.githubusercontent.com/tommantonela/sistemasRecomendacion2019/master/User-User%20Collaborative%20Filtering%20-%20movie-row.csv'
df = pd.read_csv(url, sep=',',index_col=0)

print(str(df.shape))
df.head()


Dado que esta alternativa se basa en encontrar los usuarios más similares para luego realizar las recomendaciones, vamos a encontrar los vecinos más cercanos. Para ello, debemos definir:

* La cantidad de vecinos a incorporar en la comparación.
* Cómo se definirá la semejanza entre los usuarios.

Con estas dos restricciones, vemos que tenemos un trade-off al decidir la cantidad de vecinos. Si el número de vecinos a seleccionar es demasiado bajo, lo más probable es que los usuarios no hayan rateado lo mismo, no pudiendo así proporcionar predicciones confiables. Por el contrario, si seleccionamos muchos vecinos, se incluirán vecinos "no tan similares" en la comparación, con gustos diferentes a los del usuario al que queremos darle las recomendaciones.
  
Existen [estudios](https://grouplens.org/site-content/uploads/evaluating-TOIS-20041.pdf) que determinaron que para la mayoría de las aplicaciones, considera entre $20$ y $30$ vecinos permite alcanzar los resultados óptimos.

En lo que respecta a la semejanza, existen varias alternativas, como por ejemplo, distancia Euclídea (y variaciones), cosine similarity (y variaciones), pearson, entre otras. Por ahora, vamos a utilizar la correlación de pearson que se [demostró](https://grouplens.org/site-content/uploads/evaluating-TOIS-20041.pdf) que permite alcanzar buenos resultados para este tipo de recomendaciones.

En resumen, para estas recomendaciones lo que vamos a hacer es:

1. Partiendo de la matriz `user x movie`, calcular la semejanza entre todos los usuarios.
2. Para cada usuario, filtrar los $k$ vecinos más similares.
3. Predecir los ratings basados en los vecinos seleccionados.

In [0]:
# calcular correlación Pearson entre los usuarios
df_corr = df.corr(method = 'pearson') # pandas ya permite hacer los cálculos de forma integrada
df_corr.head()

In [0]:
# buscamos los usuarios más similares/correlacionados, con el cuidado de eliminar a si mismo

def findKNearestUsers(userCorrCol, k = 5):
    return userCorrCol[userCorrCol.index != userCorrCol.name].nlargest(n = k).index.tolist()

kNeighboors = df_corr.apply(lambda col: findKNearestUsers(col))
kNeighboors

### Predicción de los ratings

Una vez que encontramos a los vecinos más similares, tenemos que predecir los ratings para los elementos que los usuarios no hayan todavía evaluado.

Una forma sencilla de hacerlo es seleccionar a todos los vecinos que hayan rateado un elemento particular y calcular el promedio de dichos ratings. En este contexto, es preciso considerar que pueden existir distintos niveles de semejanza con los vecinos. Es así como sería deseable que los vecinos con mayor semejanza tengan un peso mayor en el cálculo del rating. Considerando un caso extremo, si solo existiesen $5$ usuarios compartiendo dos reviews de productos y uno de ellos no se encuentra relacionado con el usurio de interes, aún cuando pueda ser considerado un "vecino" sería deseable que tenga un peso mínimo en el cálculo del promedio. La forma de hacer esto es calcular un promedio ponderado donde $r$ representa los ratings de los vecinos del elemento de interés y $w$ la semejanza con el usuario de interés:

$$\frac{\sum_{n=1}^{k} r_{n}w_{n}}{\sum_{n=1}^{k} w_{n}}$$
  
El promedio tradicional es simplemente un promedio ponderado donde en todos los casos $w = 1$.

Para simplificar los cálculos, se calcularán los ratings para todas las películas. En un escenario real, se haría solo para los ratings faltantes.
.


In [0]:
def calculatePredictionsUser(kNeighboors, user_correlations, df):
    
    def calculatePredictionsUserMovie(kNeighboors, user_correlations, movieRowRatings): 
        hasRatedMovie = ~np.isnan(movieRowRatings)
        if(np.sum(hasRatedMovie) != 0): # only return value if there was at least one neighboor who also rated that movie
            return np.dot(movieRowRatings.loc[hasRatedMovie], user_correlations.loc[hasRatedMovie])/np.sum(user_correlations[hasRatedMovie])
        else:
            return np.nan
        
    # looking at one user, apply function for each row = movie and predict rating for that movie
    return df.apply(lambda movieRow: calculatePredictionsUserMovie(kNeighboors, user_correlations, movieRow[kNeighboors]), axis=1)
    

####### Starting process point
# call function sending user's neighboors, neighboors similarity and movie ratings df     
moviePredictions = df.apply(lambda userRatings: calculatePredictionsUser(kNeighboors[userRatings.name], 
                                                      df_corr[userRatings.name][kNeighboors[userRatings.name]],
                                                      df))

Vamos a mirar las predicciones para un usuario:

In [0]:
moviePredictions['3867'].sort_values(ascending=False).head(10) # mirando las predicciones para un usuario random

La correlación de Pearson evalúa cuán linealmente dependientes son dos usuarios y no con qué intensidad. Esto implica que el rating entre los usuarios $U_{1} = [3,3,3,3,3]$ y $U_{2} = [4,4,4,4,4]$ y entre los usuarios $U_{3} = [2,2,2,2,2]$ y $U_{4} = [5,5,5,5,5]$ sería el mismo. Esto significa que no es posible promediar los ratings entre los usuarios debido a que la correlación no tiene en cuenta las variabilidades de las escalas entre los usuarios. 

Para tener en cuenta estas diferencias, es posible mejorar el promedio ponderando para considerar en cuánto se desvía la calificación dada por un vecino a un elemento del promedio de sus ratings. Finalmente, este valor debe ser incorporado al promedio del usuario de interés.

$$\bar{r_{u}} + \frac{\sum_{n=1}^{k} (r_{n} - \bar{r_{n}})w_{n}}{\sum_{n=1}^{k} w_{n}}$$

En la implmentación, vamos a agregar dos parámetros extra:

- `userMeanRating`. El rating promedio para el usuario particular.
- `neighboorsMeanRating`. Promedio para todos los vecinos de un usuario particular.

In [0]:
def calculatePredictionsUserNorm(kNeighboors, user_correlations, userMeanRating, neighboorsMeanRating, df):
    
    def calculatePredictionsUserMovieNorm(kNeighboors, user_correlations, userMeanRating, neighboorsMeanRating, movieRowRatings): 
        hasRatedMovie = ~np.isnan(movieRowRatings)
        if(np.sum(hasRatedMovie) != 0): # only return value if there was at least one neighboor who also rated that movie
            userRatingDeviation = movieRowRatings.loc[hasRatedMovie] - neighboorsMeanRating.loc[hasRatedMovie]
            numerator = np.dot(userRatingDeviation, user_correlations.loc[hasRatedMovie])
            return userMeanRating + numerator/np.sum(user_correlations[hasRatedMovie])
        else:
            return np.nan
        
    # looking at one user, apply function for each row = movie and predict rating for that movieprint
    return df.apply(lambda movieRow: calculatePredictionsUserMovieNorm(kNeighboors, 
                                                                       user_correlations,
                                                                       userMeanRating,
                                                                       neighboorsMeanRating,
                                                                       movieRow[kNeighboors]), axis=1)
    

####### Starting process point

meanRatingPerUser = df.apply(np.mean)

# call function sending user's neighboors, neighboors similarity and movie ratings df     
moviePredictionsNorm = df.apply(lambda userRatings: 
                                          calculatePredictionsUserNorm(kNeighboors[userRatings.name], 
                                                      df_corr[userRatings.name][kNeighboors[userRatings.name]],
                                                      np.mean(userRatings),                 
                                                      meanRatingPerUser[kNeighboors[userRatings.name]],
                                                      df))

moviePredictionsNorm['3867'].sort_values(ascending=False).head(10)

#### Comparemos los dos resultados obtenidos!

In [0]:
finalMovie = pd.DataFrame()

finalMovie['TitleNotNorm'] = moviePredictions['3867'].sort_values(ascending=False).head(10).index
finalMovie['withoutNormalisation'] = moviePredictions['3867'].sort_values(ascending=False).head(10).values
finalMovie['TitleNorm'] = moviePredictionsNorm['3867'].sort_values(ascending=False).head(10).index
finalMovie['normalised'] = moviePredictionsNorm['3867'].sort_values(ascending=False).head(10).values
finalMovie

### Algunos resultados raros...

#### Predicciones mayores que el máximo de la escala?

Al considerar puntajes normalizados, esto puede suceder si los usuarios que están siendo evaluados ya ratean las películas con un rating promedio elevado, y luego se le suma el desvío promedio de los vecinos, los cuales pueden o no estar en la misma escala.

Por ejemplo:


In [0]:
print('Promedio para el usuario 3867: ' + str(df[['3867']].apply(np.mean).values[0]))

#########
neighboors = kNeighboors['3867']
weights = df_corr[['3867']].loc[neighboors]
means = df[neighboors].apply(np.mean)
ratings = df.loc[['1891: Star Wars: Episode V - The Empire Strikes Back (1980)']][neighboors]
existingRatings = list(~(ratings.apply(np.isnan).values[0]))

# weighted average deviation
denominator = np.dot(ratings.loc[:,existingRatings] - means[existingRatings], weights[existingRatings]).tolist()[0]
avgWeightedDeviation = (denominator/np.sum(weights[existingRatings])).values[0]

print('Cuánto se desvía la media de los vencinos respecto a su propoio promedio ' +
      'Star Wars: Episode V - The Empire Strike: ' + str(avgWeightedDeviation))

Como puede observarse, el usuario `3687` no tiene un promedio alto, pero tiene vecinos que tienen promedios menores y Star Wars recibió ratings por encima de su propio promedio, lo que hizo que la predicción para `3687` superase el máximo de la escala.

#### Las películas recomendadas no son las mismas?

En los scores normalizados, Fargo aparece en el 4to lugar, mientras que no apareció al considerar los scores no normalizados. Qué puede haber pasado?

In [0]:
print('Promedio para el usuario 3867: ' + str(df[['3867']].apply(np.mean).values[0]))

#########
neighboors = kNeighboors['3867']
weights = df_corr[['3867']].loc[neighboors]
means = df[neighboors].apply(np.mean)
ratings = df.loc[['275: Fargo (1996)']][neighboors]
existingRatings = list(~(ratings.apply(np.isnan).values[0]))

print('Cuantos vecinos ratearon esta película: ' + str(np.sum(existingRatings)))
print('Ratings de los vecinos: ' + str(ratings.loc[:,existingRatings].values[0][0]))

weightedAvg = float((ratings.loc[:,existingRatings].values * weights[existingRatings]).iloc[:,0].values[0]/np.sum(weights[existingRatings]))
print('--- Predicción final para el promedio ponderado "normal": ' + str(weightedAvg))

# weighted average deviation
denominator = np.dot(ratings.loc[:,existingRatings] - means[existingRatings], weights[existingRatings]).tolist()[0]
avgWeightedDeviation = (denominator/np.sum(weights[existingRatings])).values[0]

print('\nDevío de los vecinos respecto a su propia media ' +
      'Fargo (1996): ' + str(avgWeightedDeviation))
print('--- Predicción final para el promedio ponderado normalizado: ' + str(df[['3867']].apply(np.mean).values[0] + avgWeightedDeviation))


Estos cálculos fueron ligeramente más complicados dado que se quería comparar cómo las variantes de los promedios ponderados crearon diferentes ratings para una misma película. Como puede observarse, en la predicción normalizada solo se contaba con un vecino que había visto la película y este vecino la rateo con más de un punto por encima de su propio promedio. Entonces, la predicción era buena, pero no fue tan buena como el promedio ponderado normal, dado que solo fue buena para un único vecino, y no considerando la escala completa de rating.


### Resumiendo...

User-User collaborative filtering ofrece una mejora respecto de los recomendadores no personalizados y basados en contenido. Ahora es posible realizar recomendaciones personalizadas, pero sin tener el desafío de cómo caracterizar y a los elementos. Sin embargo, todavía tiene algunos problemas:

* No es escalable. 
* La semejanza entre los usuarios no se mantiene a lo largo del tiempo. 
* Dado que la mayoría de los usuarios solo ha etiquetado un reducido número de elementos, los datos son generalemente sparse.



             

## Item-Item collaborative filtering

El Item-Item CF ue creado por Amazon para resolver las problemáticas del `User-User`. En esta perfpectiva, como el nombre lo sugiere, se cambia el foco de los usuarios a los items. Entonces, en lugar de tener una matriz de semejanzas entre los usuarios, ahora se tiene una entre los elementos. Por ejemplo, cuando el usuario $u$ compró el elemento $i_{1}$, el cual era similar $i_{2}$, sería posible predecir que a $u$ le va a interesar $i_{2}$.

<p align="center">
<img src="https://miro.medium.com/max/1920/1*QvhetbRjCr1vryTch_2HZQ.jpeg" width="600">
</p>

Esta perspectiva ayuda a solucionar el problema de las pocas reviews provistas por unos pocos usuarios. Tener una gran cantidad de reviews hace que las relaciones entre los elementos no cambien demasiado, haciendo que las relaciones entre los elementos sean estables. Esto significa que la matriz de semejanzas no necesita ser recalculada seguido.

Los pasos para realizar recomendaciones son:

1. Obtener los datos.
2. Crear la matriz de semejanzas correspondientes.
3. Realizar las predicciones :D


Vamos a usar el mismo dataset que para user-user, solo que ahora, como el énfasis está puesto en los items, vamos a trasponerlo. Al igual que en el caso anterior, cada celda $c_{u,m}$ contiene el rating que el usario $u$ dió a la película $m$. Si el usuario no rateo la película, la celda correspondiente se encontrará vacía.

In [0]:
item_item = df.transpose()

item_item

### Crear la matriz de semejanzas

Al igual que para las recomendaciones User-User, se tienen diversas posibildiades para elegir cómo calcular la semejanza entre los usuarios. En este contexto, [Herlocker et al (2002)](https://grouplens.org/site-content/uploads/evaluating-TOIS-20041.pdf) realizaron una evaluación experimental y encontraron que el cosine similarity permitía en alcanzar los mejores resultados.


#### Calculando cosine similarity

Un punto importante aquí es el cálculo del denominador. Aunque hacemos el producto de punto solo con valores existentes en ambas matrices, la norma de los vectores individuales considera todos los valores, y no solo las intersecciones entre `array_1` y `array_2`.


In [0]:
def cos_similarity(item1, item2):
    item1Values = ~np.isnan(item1) # nos quedamos con los ratings existentes 
    item2Values = ~np.isnan(item2)
    allValues = np.logical_and(item1Values,item2Values) # calculamos la intersección
    return np.dot(item1[allValues], item2[allValues])/(np.linalg.norm(item1[item1Values]) * np.linalg.norm(item2[item2Values]))

def pre_cos_similarity(item1, item_item):
    return item_item.apply(lambda item2: cos_similarity(item1, item2))

item_item_corr = item_item.apply(lambda item1: pre_cos_similarity(item1, item_item))
item_item_corr.head()

### Predicción de los ratings

En este punto ya sabemos qué items son más similares a cuales. Esto ayudará a predecir un gran rating para dar pesos más altos a los ítems que son más similares a los que el usuario ya ha rateado.

$$\frac{\sum_{n=1}^{k} r_{n}w_{n}}{\sum_{n=1}^{k} w_{n}}$$
  
La diferencia con la recomendación User-User es que, al no haber más vecinos, el $n$ en la suma considera todos los ratings que el usuario $u$ haya realizado.


In [0]:
def predictRating(userRatings, itemSimilarity):
    userHasRating = ~np.isnan(userRatings)
    return np.dot(userRatings[userHasRating], itemSimilarity[userHasRating])/np.sum(itemSimilarity[userHasRating])

def pre_predictRating(userRatings, df_corr):
    return item_item_corr.apply(lambda itemSimilarity: predictRating(userRatings, itemSimilarity))

# hacemos las predicciones para cada usuario
predictions = item_item.apply(lambda userRatings: pre_predictRating(userRatings, item_item_corr), axis=1)
predictions.head()

Finalmente, una vez que estimamos los ratings que los usuarios darían a las películas que todavía no ratearon, podemos hacer las recomendaciones!


In [0]:
# volvemos a nuestra matriz donde las columnas representan los usuarios y hacemos lo mismo que en el ejemplo anterior!
user_predictions = predictions.transpose()

x = [p[0] for p in np.argwhere(df['3867'].isnull().values).tolist()]

df['3867'].index.values[x].tolist()

user_predictions['3867'].loc[df['3867'].index.values[x].tolist()].sort_values(ascending=False).head(10)


Asimismo, también podemos comparar los ratings originales que dió el usuario con aquellos que fueron estimados en la recomendación.

In [0]:
a = pd.DataFrame(df['3867'])
a['new'] = user_predictions['3867']
a[~a['3867'].isna()]
print(a)

Al igual que para la recomendación User-User es posible normalizar el promedio ponderado considerando el promedio de los elementos rateados. La ventaja es que permite considerar la variabilidad en la escala de los ratings de los diferentes usuarios. Para ello, normalizamos los ratings.

In [0]:
# mean normalise
def subtractFromMean(col, meanCol):
    result = np.array([np.nan] * col.shape[0])
    isValidValue = ~np.isnan(col)
    result[isValidValue] = col.values[isValidValue] - meanCol.values[isValidValue]
    return result
userMeanRatings = df.apply(np.mean, axis=1)
df_ratings_norm = df.apply(lambda col: subtractFromMean(col, userMeanRatings)).transpose()

print(item_item.shape)
print(df_ratings_norm.shape)

# similarity matrix
item_item_corr_norm = item_item.apply(lambda item1: pre_cos_similarity(item1, df_ratings_norm))
item_item_corr_norm.head()

def replaceNegative(col):
    col[col < 0] = 0
    return col

item_item_corr_norm = item_item_corr_norm.apply(replaceNegative)

Haciendo las recomendaciones!

Al igual que hicimos en el caso anterior, qué películas se le recomendaría a los usuarios?

In [0]:
# recomendando pelis...
predictions_norm = item_item.apply(lambda userRatings: pre_predictRating(userMeanRatings, item_item_corr_norm), axis=1)
predictions_norm.head() 

In [0]:
user_predictions_norm = predictions_norm.transpose()

x = [p[0] for p in np.argwhere(df['3867'].isnull().values).tolist()]

df['3867'].index.values[x].tolist()

user_predictions_norm['3867'].loc[df['3867'].index.values[x].tolist()].sort_values(ascending=False).head(10)

In [0]:
# comparar las predicciones con los ratings originales que había asignado el usuario
a = pd.DataFrame(df['3867'])
a['predictions_norm'] = user_predictions_norm['3867']
a[~a['3867'].isna()]
print(a)

#### Comparemos los dos resultados obtenidos!

**Tarea!** Comparar los resultados de las recomendaciones como hicimos para el user-user

In [0]:
# comparación de resultados

finalMovie = pd.DataFrame()

finalMovie['TitleNotNorm'] = user_predictions['3867'].sort_values(ascending=False).head(10).index
finalMovie['withoutNormalisation'] = user_predictions['3867'].sort_values(ascending=False).head(10).values
finalMovie['TitleNorm'] = user_predictions_norm['3867'].sort_values(ascending=False).head(10).index
finalMovie['normalised'] = user_predictions_norm['3867'].sort_values(ascending=False).head(10).values
finalMovie

### Resumiendo

La recomendación Item-Item mejora a la User-User en términos de la complejidad computacional. Algunas consideraciones:

1. Para el establecimiento de ratings estables es preciso que `number_users >> number_items`.
2. Los mejores resultados se alcanzarán cuando los ratings sean estables, es decir, cuando los elementos tienen muchas evaluaciones.
3. Falta de serendipia. Al basarse la recomendación en la semejanza de los elementos, en ocasiones puede ser difícil recomendar, items relevantes pero cuya semejanza con los ya conocimos es menor. 
